# Inlezen Ribasim model Huite en het model uitbreiden met LSW ID's

In [16]:
import geopandas as gpd
import numpy as np
import scipy.ndimage
import xarray as xr

def column_names_to_string(ds):
    ds["profile_col"] = [chr(int(v)) for v in ds["profile_col"].values]

# Read the existing (sanitized) input.
# Read the shapefiles of districts and LSW's.
# Some LSW are not part of the network. This is given by the sanitized input.
# Get the bounds of the Berkel district.

bach_ds = xr.open_dataset("c:/Users/ngoor/Projects/Repos/lhm-ribasim/data/ribasim_testmodel/bach-cases/data/1-external/input-mozart.nc")
column_names_to_string(bach_ds)

gdf_lsw = gpd.read_file("c:/Users/ngoor/Projects/Repos/lhm-ribasim/data/lhm4.3/coupling/lsws.shp")
lsw_relevant = bach_ds["node"].values.astype(int)
gdf_lsw = gdf_lsw[gdf_lsw["LSWFINAL"].isin(lsw_relevant)]
gdf_lsw["lsw_area"] = gdf_lsw.area

# Specify the output file path relevant LSWs
output_file_path = "lsw_relevant.shp"

# Save the unjoined LSWs to a shapefile
gdf_lsw.to_file(output_file_path)

print("Unjoined LSWs saved to:", output_file_path)



Unjoined LSWs saved to: lsw_relevant.shp


In [ ]:
import geopandas as gpd

# Read the nodes file and model file
nodes_gdf = gpd.read_file("nodes.gpkg")
model_gdf = gpd.read_file("../data/ribasim_testmodel/model.gpkg", layer='Node', index_col='fid')

filtered_model_gdf = model_gdf[model_gdf['type'] == 'Basin']

# Perform the spatial join
joined_gdf = gpd.sjoin_nearest(filtered_model_gdf, nodes_gdf, how='left')

# Reset the index of the joined GeoDataFrame to preserve the original 'fid' values
joined_gdf.reset_index(drop=True, inplace=True)

# Set the value of 'lswfinal' to None for non-matching records
joined_gdf.loc[model_gdf['type'] != 'Basin', 'lswfinal'] = None

# Add a new column 'fid' to the joined GeoDataFrame using the index
joined_gdf['fid'] = joined_gdf.index

# Rename one of the 'TYPE' columns to avoid duplicate column names
joined_gdf.rename(columns={'TYPE': 'TYPE_2'}, inplace=True)

# Specify the output file path for the modified joined GeoDataFrame
output_file_path = "modified_joined_gdf.gpkg"

# Save the modified joined GeoDataFrame to a GeoPackage
joined_gdf.to_file(output_file_path, driver='GPKG')
